In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data_path = "./data/HealthCare_Gold_datasets/Context/full_role_based_metric_access.json"
with open(data_path, "r") as f:
    data = json.load(f)
df = pd.DataFrame(data)

non_clinical_staff_df = df['non_clinical_staff']['metrics']
manager_df = df['managers']['metrics']
admin_df = df['administrative_staff']['metrics']

In [3]:
non_clinical_meta_path = "./data/HealthCare_Gold_datasets/Context/non_clinical_metrics_metadata.json"
with open(non_clinical_meta_path, "r") as f:
    non_clinical_meta = json.load(f)

In [4]:
## Join non_clinical staff with manager and admin metrics
non_clinical_staff_df = pd.DataFrame(non_clinical_staff_df)
manager_df = pd.DataFrame(manager_df)
admin_df = pd.DataFrame(admin_df)
# Merging non-clinical staff metrics with manager and admin metrics
merged_df = pd.concat([non_clinical_staff_df, manager_df, admin_df], ignore_index=True)
# Display the merged DataFrame
merged_df.head()

,Column Name,Data Type,Description,Granularity,Calculation,source_sheet
0,patient_arrival_count_monthly,int64,Total number of patient arrivals in the month,Monthly,Sum of patient arrivals for the month,emergency_department_metrics
1,patient_arrival_count_daily,object,Daily count of patient arrivals (JSON format),Daily (nested),"Aggregated from daily patient logs, may includ...",emergency_department_metrics
2,bed_utilization_rate,float64,Emergency bed utilization rate (%),Monthly,(Beds in Use / Total Beds) * 100,emergency_department_metrics
3,staff_utilization_rate,float64,ED staff utilization rate (%),Monthly,(Staff in Use / Total Staff) * 100,emergency_department_metrics
4,outpatient_visit_count_monthly,int,Total number of outpatient visits in the month,Monthly,Sum of daily outpatient visit counts,outpatient_department_metrics


In [5]:
table_name_mapping = {'emergency_department_metrics':'emergency_metrics',
 'inpatient_ward_metrics': 'inpatient_metrics',
 'outpatient_department_metrics': 'outpatient_metrics',
 'surgery_department_metrics': 'surgery_metrics',
 'radiology_department_metrics': 'radiology_metrics',
 'laboratory_department_metrics': 'lab_metrics',
 'pharmacy_department_metrics': 'pharmacy_metrics',
 'icu_metrics': 'icu_metrics',
 'administration_metrics': 'hospital_administration_metrics',
 'quality_safety_metrics': 'patient_safety_metrics',
 'finance_billing_metrics' : 'financial_metrics',
 'patient_experience_metrics': 'patient_experience_metrics'}

column_categories = ["Identifier","Date/Timestamp","Categorical","Description","Flag","Metric"]
keys_req = ["type","table_name","column_name","data_type","granularity","calculation","description","roles_allowed"]

In [6]:
non_clinical_staff_df['role'] = 'non_clinical_staff'
manager_df['role'] = 'manager'
admin_df['role'] = 'administrative_staff'
# Concatenate the DataFrames
all_roles_df = pd.concat([non_clinical_staff_df, manager_df, admin_df], ignore_index=True)

In [7]:
## from all roles df,create a new df called unique_combination that has 
#  for a unique combination of 'source sheet' and 'Column Name',create a new column called roles which is a list of roles that have access to that metric
unique_combination = all_roles_df.groupby(['source_sheet', 'Column Name'])['role'].apply(list).reset_index()
# Rename the columns for clarity
unique_combination.columns = ['source_sheet', 'Column Name', 'roles']
# Display the unique combinations DataFrame
unique_combination.head()

,source_sheet,Column Name,roles
0,administration_metrics,avg_staff_overtime_hours,[manager]
1,administration_metrics,billing_accuracy_rate,[manager]
2,administration_metrics,claims_processing_time_days,[manager]
3,administration_metrics,compliance_audit_score,"[manager, administrative_staff]"
4,administration_metrics,cost_per_case,[manager]


In [8]:
def determine_col_type(column_name,data_type):
    dtype_str = str(data_type).lower()
    if "_id" in column_name.lower() or "_code" in column_name.lower():
        return "Identifier"
    elif "date" in column_name.lower() or "timestamp" in column_name.lower():
        return "Date/Timestamp"
    elif "flag" in column_name.lower() or dtype_str == "boolean" or dtype_str == "bool":
        return "Flag"
    elif "type" in column_name.lower() or "category" in column_name.lower():
        return "Categorical"
    else:
        return "Metric"

In [9]:
non_clinical_metadata = []
for table_key,cols in non_clinical_meta.items():
    # print(vals)
    # break
    table_name_cleaned = table_key.strip()
    # print(table_name_cleaned)
    # print(len(cols))
    
    for col in cols:   
        meta_info = {key: None for key in keys_req}
        # print(col['Column Name'])
        meta_info['type'] = determine_col_type(col['Column Name'], col['Data Type'])
        meta_info['table_name'] = table_name_mapping.get(table_name_cleaned)
        meta_info['column_name'] = col['Column Name'].strip()
        meta_info['data_type'] = col['Data Type'].strip()
        meta_info['granularity'] = col.get('Granularity', '').strip()
        meta_info['calculation'] = col.get('Calculation', '').strip()
        meta_info['description'] = col.get('Description', '').strip()

        meta_info['roles_allowed'] = unique_combination[(unique_combination['source_sheet'] == table_name_cleaned) & 
                                                        (unique_combination['Column Name'] == col['Column Name'].strip())]['roles'].tolist()
        # remove nested lists and convert to a simple list
        if isinstance(meta_info['roles_allowed'], list) and len(meta_info['roles_allowed']) == 1:
            meta_info['roles_allowed'] = meta_info['roles_allowed'][0]
        if not meta_info['roles_allowed']:
            meta_info['roles_allowed'] = []
        non_clinical_metadata.append(meta_info)       

    # break
    # logic for determining column type
    

In [10]:
# Load CSVs
departments_df = pd.read_csv("./data/HealthCare_Gold_datasets/Hospital info/Departments.csv")
hospital_df = pd.read_csv("./data/HealthCare_Gold_datasets/Hospital info/Hospitals.csv")
staff_df = pd.read_csv("./data/HealthCare_Gold_datasets/Hospital info/Staff.csv")

# Assign roles
departments_df['role'] = 'manager'
hospital_df['role'] = 'manager'
staff_df['role'] = 'manager'

# Map DataFrames to their respective table names
table_mapping = {
    "departments": departments_df,
    "hospitals": hospital_df,
    "staff": staff_df
}

# Utility to infer column type name (basic guess)
# def infer_type(dtype):
#     if pd.api.types.is_datetime64_any_dtype(dtype):
#         return "Date/Timestamp"
#     elif pd.api.types.is_numeric_dtype(dtype):
#         return "Numeric"
#     else:
#         return "String"

# Build the metadata list
column_metadata_dim = []

for table_name, df in table_mapping.items():
    for column in df.columns:
        if column == 'role':
            continue  # Skip the 'role' column we added
        col_dtype = df[column].dtype
        column_metadata_dim.append({
            "type": determine_col_type(column, col_dtype),
            "table_name": table_name,
            "column_name": column,
            "data_type": str(col_dtype),
            "granularity": None,
            "calculation": None,
            "description": f"{column.replace('_', ' ').capitalize()}",
            "roles_allowed": ["manager"]
        })

column_metadata_dim


[{'type': 'Identifier',
  'table_name': 'departments',
  'column_name': 'department_id',
  'data_type': 'object',
  'granularity': None,
  'calculation': None,
  'description': 'Department id',
  'roles_allowed': ['manager']},
 {'type': 'Identifier',
  'table_name': 'departments',
  'column_name': 'hospital_id',
  'data_type': 'object',
  'granularity': None,
  'calculation': None,
  'description': 'Hospital id',
  'roles_allowed': ['manager']},
 {'type': 'Metric',
  'table_name': 'departments',
  'column_name': 'department_name',
  'data_type': 'object',
  'granularity': None,
  'calculation': None,
  'description': 'Department name',
  'roles_allowed': ['manager']},
 {'type': 'Metric',
  'table_name': 'departments',
  'column_name': 'floor_number',
  'data_type': 'object',
  'granularity': None,
  'calculation': None,
  'description': 'Floor number',
  'roles_allowed': ['manager']},
 {'type': 'Metric',
  'table_name': 'departments',
  'column_name': 'head_of_department',
  'data_type

In [11]:
# combine the non_clinical_metadata with the column_metadata_dim
combined_metadata = non_clinical_metadata + column_metadata_dim

In [13]:
with open("column_level_metadata.json", "w") as f:
    json.dump(combined_metadata, f, indent=4)